In [1]:
%load_ext autoreload
%autoreload 2

In [98]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import torch

import tsvar

In [348]:
mu = torch.tensor([0.2, 0.1])
beta = torch.tensor([1.1, 1.5])
A = torch.tensor([
    [0.7, 0.3],
    [0.1, 0.9]
])
dim = 2

#coeffs = torch.cat((mu, beta[0], A.flatten()))

In [464]:
end_time = 4000.0

wold_sim = tsvar.simulate.GrangeBuscaSimulator(mu_rates=mu, Alpha_ba=A, Beta_b=beta)
events = wold_sim.simulate(end_time)
events = [torch.tensor(ev, dtype=torch.float) for ev in events]

print(f"Simulated {sum(map(len, events))} events with end time: {end_time}")
print(list(map(len, events)))

Simulated 4144 events with end time: 4000.0
[2146, 1998]


In [465]:
class WoldModelSimple(tsvar.wold_model.WoldModel):
    
    def set_data(self, events, end_time=None):
        super().set_data(events, end_time)
        self.n_params = self.dim * (self.dim - 1) + self.dim

    def log_likelihood(self, coeffs):
        # Extract each set of parameters
        mu = coeffs[:self.dim]
        beta = coeffs[self.dim:2*self.dim]
        alpha = coeffs[2*self.dim:].reshape(self.dim, self.dim)
        # Compute the log-likelihood
        log_like = 0
        for i in range(self.dim):
            # Compute the intensity at each event
            lam_ik_arr = mu[i] + torch.sum(
                self.valid_mask_ikj[i] * alpha[:,i] /(beta.unsqueeze(0) + self.delta_ikj[i]), 
                axis=1)
            # Add the log-intensity term
            log_like += lam_ik_arr[:-1].log().sum()
            # Subtract the integral term
            log_like -= lam_ik_arr[0] * self.events[i][0]
            log_like -= torch.sum(lam_ik_arr[1:] * (self.events[i][1:] - self.events[i][:-1]))
        return log_like

In [473]:
model = tsvar.wold_model.WoldModel()
model.set_data(events, end_time)

coeffs_true = torch.cat((mu, beta, A.flatten()))

x0 = torch.cat((
    0.01 * torch.ones(dim, dtype=torch.float),
    1.0 * torch.ones(dim, dtype=torch.float),
    0.5 * torch.ones((dim, dim), dtype=torch.float).flatten()
))

C = 1000.0 * torch.ones(len(coeffs_true))
prior = tsvar.priors.GaussianPrior(dim=None, n_params=len(coeffs_true), C=C)

optimizer = torch.optim.Adam([x0], lr=0.05)

learner = tsvar.learners.MLELearner(
    model=model, 
    prior=prior, 
    optimizer=optimizer,
    tol=1e-10,
    max_iter=100000,
    debug=False)

learner_callback = tsvar.utils.callbacks.LearnerCallbackMLE(x0, coeffs_true.numpy(), print_every=10)

In [474]:
coeffs_true

tensor([0.2000, 0.1000, 1.1000, 1.5000, 0.7000, 0.3000, 0.1000, 0.9000])

In [475]:
x0

tensor([0.0100, 0.0100, 1.0000, 1.0000, 0.5000, 0.5000, 0.5000, 0.5000])

In [476]:
coeffs_hat = learner.fit(events, end_time, x0, callback=learner_callback)

iter:   650 | dx: 1.19e-07 | loss: 6.67e+03    loss = 6673.4716796875 Converged!


In [477]:
coeffs_hat.detach().numpy().round(2)

array([0.24, 0.09, 1.01, 1.15, 0.64, 0.28, 0.06, 0.77], dtype=float32)

In [478]:
coeffs_true.detach().numpy().round(2)

array([0.2, 0.1, 1.1, 1.5, 0.7, 0.3, 0.1, 0.9], dtype=float32)

In [479]:
-model.log_likelihood(x0)

tensor(6833.5337)

In [480]:
-model.log_likelihood(coeffs_hat)

tensor(6673.4683, grad_fn=<NegBackward>)

In [481]:
-model.log_likelihood(coeffs_true)

tensor(6677.8965)

RuntimeError: Can't call numpy() on Variable that requires grad. Use var.detach().numpy() instead.